<h3>Combination of reinforcement learning (RL) and heuristic methods to optimize the packing sequence</h3>

<h5>Machine Learning-Based Approach for 2D Bin Packing Problem</h5>
<p>Hybrid Approach: Reinforcement Learning + Heuristic Methods</p>

In [1]:
# import sys
# print(sys.executable)


In [2]:
# pip install gym
# !pip install stable_baselines3
# print(sys.executable)
# !pip uninstall stable_baselines3
# !pip install stable_baselines3
# !pip install stable_baselines3[extra]
# %pip install stable-baselines3


In [3]:
# Import Libraries
import numpy as np
import csv
from typing import List, Dict, Tuple
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env

# Visualization (Optional)
import matplotlib.pyplot as plt
import matplotlib.patches as patches

<h3>2D Bin Packing Environment</h3>

In [4]:
# Define the 2D Bin Packing Environment
class BinPackingEnv(gym.Env):
    def __init__(self, pieces, bin_sizes):  # Changed back to bin_sizes for clarity
        super(BinPackingEnv, self).__init__()
        self.pieces = pieces
        self.bin_sizes = bin_sizes  # Store all available bin sizes
        self.current_bin_index = 0  # Start with the first bin
        self.current_bin_size = self.bin_sizes[self.current_bin_index]  # Changed to current_bin_size
        self.n_pieces = len(pieces)
        
        # Fix the max_bin_dimension calculation
        max_bin_dimension = max(max(size[0], size[1]) for size in self.bin_sizes)
        
        self.action_space = spaces.Discrete(self.n_pieces)
        self.observation_space = spaces.Box(
            low=0, 
            high=np.array([max_bin_dimension] * 2 * len(self.pieces)).reshape(len(self.pieces), 2), 
            shape=(len(self.pieces), 2), 
            dtype=float
        )
        self.reset()

    def switch_to_next_bin(self):
        if self.current_bin_index < len(self.bin_size) - 1:
            self.current_bin_index += 1
            self.bin_size = self.bin_sizes[self.current_bin_index]  # Update to the next bin
            print(f"Switched to new bin: {self.bin_size}")
        else:
            print("No more bins available.")

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_bin = np.zeros(self.current_bin_size)  # Use current_bin_size here
        self.packed_pieces = []
        self.state = np.array([[piece[0], piece[1]] for piece in self.pieces], dtype=np.float32)
        self.reward = 0
        info = {}
        return self.state, info

    def step(self, action):
        piece = self.pieces[action]
        if self.can_place(piece):
            self.place_piece(piece)
            self.packed_pieces.append(piece)
            self.state[action, 0] = piece[0]  # Update length
            self.state[action, 1] = piece[1]  # Update width
            self.reward += 1
        else:
            self.reward -= 1  # Penalty for not placing
    
        done = len(self.packed_pieces) == self.n_pieces
        terminated = done  # Use `terminated` to indicate the end of an episode
        truncated = False  # Use `truncated` to indicate if the episode was truncated
    
        info = {}  # Additional information
    
        return self.state, self.reward, terminated, truncated, info

    def render(self, mode='human'):
        print(f"Packed Pieces: {self.packed_pieces}")

    def can_place(self, piece):
        # Simple heuristic to check if the piece can fit in the current bin
        return piece[0] <= self.bin_size[0] and piece[1] <= self.bin_size[1]

    # CoPilot
    def can_place(self, piece):
        # Simple heuristic to check if the piece can fit in the current bin
        return piece[0] <= self.current_bin_size[0] and piece[1] <= self.current_bin_size[1]

    def place_piece(self, piece):
        # Place the piece in the bin (simplified for illustration)
        self.current_bin[:piece[0], :piece[1]] = 1


In [5]:
gap = 0  # Gap between parts in mm

<h3>Define file paths</h3>

In [6]:
# Define file paths
glass_data_file = 'data/glass_data.csv'

# Load Data Functions
def load_glass_data(filepath: str) -> List[Dict]:
    with open(filepath, 'r') as file:
        reader = csv.DictReader(file)
        return [{'location': row['location'], 
                 'length': int(row['glass_length']), 
                 'height': int(row['glass_height']), 
                 'qty': int(row['glass_qty'])} for row in reader]

# Convert dictionary data to tuple and expand based on quantity
def pieces_dict_to_expanded_tuples(data: List[Dict]) -> List[Tuple]:
    pieces_expanded_tuples = []
    for item in data:
        for _ in range(item['qty']):
            pieces_expanded_tuples.append((item['length'], item['height']))
    return pieces_expanded_tuples

def calculate_total_area(pieces: List[Tuple[int, int]]) -> int:
    return sum((length / 1000) * (height / 1000) for length, height in pieces)

# Load the data
pieces = load_glass_data(glass_data_file)

# Convert to expanded tuples
pieces = pieces_dict_to_expanded_tuples(pieces)  # Changed variable name to pieces

# Calculate the total area of all pieces
total_area = calculate_total_area(pieces)

# Print the total area
print(f"Total area of all Glass: {total_area:.3f} Sqm")

# Add this line after pieces is created
print(f"Number of Glass: {len(pieces)}")

# Simply print the pieces list
print(pieces)

Total area of all Glass: 451.142 Sqm
Number of Glass: 260
[(973, 1883), (973, 1883), (973, 1883), (973, 1883), (973, 1883), (973, 1883), (973, 1608), (973, 1608), (973, 1608), (973, 1608), (973, 1608), (973, 1608), (973, 1608), (973, 1608), (973, 1608), (973, 1608), (973, 1608), (973, 1608), (976, 2358), (976, 2358), (976, 2358), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (952, 1738), (1010, 1594), (1010, 1594), (1010, 1594), (1010, 1594), (1010, 1594), (1010, 1594),

In [7]:
# Define file paths
stock_sizes_file = 'data/glass_sheet_size1.csv'

# Load Data Functions
def load_stock_sizes(filepath: str) -> List[Dict]:
    with open(filepath, 'r') as file:
        reader = csv.DictReader(file)
        return [{'length': int(row['length']), 
                 'width': int(row['width']), 
                 'qty': int(row['qty'])} for row in reader]

# define the get_bin_size function
def get_bin_size(data: List[Dict]) -> List[Tuple[int, int]]:
    return [(item['length'], item['width']) for item in data]

def bins_dict_to_tuples(data: List[Dict]) -> List[Tuple]:
    bins_expanded_tuples = []
    for item in data:
        for _ in range(item['qty']):
            bins_expanded_tuples.append((item['length'], item['width']))
    return bins_expanded_tuples

def calculate_total_area(bins: List[Tuple[int, int]]) -> int:
    return sum((length / 1000) * (width / 1000) for length, width in bins)

# Load the data
bins = load_stock_sizes(stock_sizes_file)

# Convert to expanded tuples
bins = bins_dict_to_tuples(bins)  # Changed variable name to bins

# Calculate the total area of all bins
total_area = calculate_total_area(bins)

# Print the total area
print(f"Total area of all bins: {total_area:.3f} Sqm")

# Add this line after bins is created
print(f"Number of bins: {len(bins)}")

# Simply print the bins list
print(bins)


Total area of all bins: 2009.520 Sqm
Number of bins: 300
[(2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (2438, 2100), (243

<h3>Heuristic Placement Strategy (Best Fit Decreasing)</h3>

In [8]:
# Heuristic Placement Strategy (Best Fit Decreasing)
def best_fit_decreasing(pieces, bin_size):
    # Take the first bin size as default if bin_size is a list
    default_bin_size = bin_size[0] if isinstance(bin_size, list) else bin_size

    bins = []
    for piece in sorted(pieces, key=lambda x: max(x), reverse=True):
        placed = False
        for bin in bins:
            if bin['remaining_width'] >= piece[0] and bin['remaining_height'] >= piece[1]:
                bin['placements'].append(piece)
                bin['remaining_width'] -= piece[0]
                bin['remaining_height'] -= piece[1]
                placed = True
                break
        if not placed:
            new_bin = {
                'size': bin_size,
                'remaining_width': bin_size[0] - piece[0],
                'remaining_height': bin_size[1] - piece[1],
                'placements': [piece]
            }
            bins.append(new_bin)
    return bins


<h3>Create a sample dataset of pieces and bin size</h3>

In [9]:
# Create a sample dataset of pieces and bin size
pieces_raw = load_glass_data(glass_data_file)
bin_size_raw = load_stock_sizes(stock_sizes_file)

# Convert pieces to expanded tuples
pieces = pieces_dict_to_expanded_tuples(pieces_raw)

# Convert bin_size to tuples
bin_size = get_bin_size(bin_size_raw)

# Initialize the environment
env = BinPackingEnv(pieces, bin_size)
check_env(env)

D:\Python311\Lib\site-packages\stable_baselines3\common\env_checker.py:272: UserWarning: Your observation  has an unconventional shape (neither an image, nor a 1D vector). We recommend you to flatten the observation to have only a 1D vector or use a custom policy to properly process the data.
  warnings.warn(


In [10]:
# Train the model
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 331      |
|    ep_rew_mean     | 3.12e+04 |
| time/              |          |
|    fps             | 582      |
|    iterations      | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 2048     |
---------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 330           |
|    ep_rew_mean          | 3.16e+04      |
| time/                   |               |
|    fps                  | 442           |
|    iterations           | 2             |
|    time_elapsed         | 9             |
|    total_timesteps      | 4096          |
| train/                  |               |
|    approx_kl            | 0.00017091754 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2       

In [11]:
# Test the trained model
obs, info = env.reset()  # Get the observation and info
for _ in range(len(pieces)):
    action, _states = model.predict(obs)
    obs, rewards, terminated, truncated, info = env.step(action)
    env.render()
    if terminated or truncated:
        break


Packed Pieces: [(964, 1603)]
Packed Pieces: [(964, 1603), (1010, 1594)]
Packed Pieces: [(964, 1603), (1010, 1594), (1010, 1594)]
Packed Pieces: [(964, 1603), (1010, 1594), (1010, 1594), (964, 1603)]
Packed Pieces: [(964, 1603), (1010, 1594), (1010, 1594), (964, 1603), (964, 1902)]
Packed Pieces: [(964, 1603), (1010, 1594), (1010, 1594), (964, 1603), (964, 1902), (1010, 1594)]
Packed Pieces: [(964, 1603), (1010, 1594), (1010, 1594), (964, 1603), (964, 1902), (1010, 1594), (964, 1603)]
Packed Pieces: [(964, 1603), (1010, 1594), (1010, 1594), (964, 1603), (964, 1902), (1010, 1594), (964, 1603), (964, 1603)]
Packed Pieces: [(964, 1603), (1010, 1594), (1010, 1594), (964, 1603), (964, 1902), (1010, 1594), (964, 1603), (964, 1603)]
Packed Pieces: [(964, 1603), (1010, 1594), (1010, 1594), (964, 1603), (964, 1902), (1010, 1594), (964, 1603), (964, 1603), (952, 1738)]
Packed Pieces: [(964, 1603), (1010, 1594), (1010, 1594), (964, 1603), (964, 1902), (1010, 1594), (964, 1603), (964, 1603), (952, 

<h3>Apply heuristic placement strategy</h3>

In [12]:
# Apply heuristic placement strategy
bins = best_fit_decreasing(pieces, bin_size[0])
print("Bins after heuristic placement:")
for i, bin in enumerate(bins):
    print(f"Bin {i+1}: {bin['placements']}")

# def plot_bin(bin):
#     fig, ax = plt.subplots()
#     for piece in bin['placements']:
#         rect = patches.Rectangle((0, 0), piece[0], piece[1], linewidth=1, edgecolor='r', facecolor='b')
#         ax.add_patch(rect)
#     plt.xlim(0, bin['size'][0])
#     plt.ylim(0, bin['size'][1])
#     plt.show()

# for bin in bins:
#     plot_bin(bin)

Bins after heuristic placement:
Bin 1: [(976, 2358)]
Bin 2: [(976, 2358)]
Bin 3: [(976, 2358)]
Bin 4: [(967, 2328)]
Bin 5: [(967, 2328)]
Bin 6: [(967, 2328)]
Bin 7: [(967, 2328)]
Bin 8: [(967, 2328)]
Bin 9: [(967, 2328)]
Bin 10: [(967, 2328)]
Bin 11: [(967, 2328)]
Bin 12: [(967, 2328)]
Bin 13: [(967, 2328)]
Bin 14: [(967, 2328)]
Bin 15: [(967, 2328)]
Bin 16: [(967, 2328)]
Bin 17: [(967, 2328)]
Bin 18: [(967, 2328)]
Bin 19: [(967, 2328)]
Bin 20: [(967, 2328)]
Bin 21: [(967, 2328)]
Bin 22: [(967, 2328)]
Bin 23: [(967, 2328)]
Bin 24: [(967, 2328)]
Bin 25: [(954, 2268)]
Bin 26: [(954, 2268)]
Bin 27: [(954, 2268)]
Bin 28: [(954, 2268)]
Bin 29: [(954, 2268)]
Bin 30: [(954, 2268)]
Bin 31: [(954, 2268)]
Bin 32: [(951, 2263)]
Bin 33: [(951, 2263)]
Bin 34: [(951, 2263)]
Bin 35: [(951, 2263)]
Bin 36: [(951, 2263)]
Bin 37: [(951, 2263)]
Bin 38: [(951, 2263)]
Bin 39: [(951, 2218)]
Bin 40: [(951, 2218)]
Bin 41: [(951, 2218)]
Bin 42: [(951, 2218)]
Bin 43: [(951, 2218)]
Bin 44: [(951, 2218)]
Bin 45: [

In [13]:
def analyze_bin_usage(bins):
    """Analyze bin usage and return detailed metrics"""
    total_bins = len(bins)
    bin_area_used = 0
    bin_area_total = 0
    bin_groups = {}
    
    for i, bin in enumerate(bins, 1):
        # Calculate areas
        bin_size = bin['size']
        total_bin_area = (bin_size[0] / 1000) * (bin_size[1] / 1000)  # Convert to sqm
        bin_area_total += total_bin_area
        
        # Calculate used area in this bin
        pieces_area = sum((piece[0] / 1000) * (piece[1] / 1000) for piece in bin['placements'])
        bin_area_used += pieces_area
        
        # Group bins by size
        bin_key = f"{bin_size[0]}x{bin_size[1]}"
        if bin_key not in bin_groups:
            bin_groups[bin_key] = {
                'count': 0,
                'pieces': [],
                'total_area': 0,
                'used_area': 0
            }
        bin_groups[bin_key]['count'] += 1
        bin_groups[bin_key]['pieces'].extend(bin['placements'])
        bin_groups[bin_key]['total_area'] += total_bin_area
        bin_groups[bin_key]['used_area'] += pieces_area
        
    # Calculate wastage
    wastage_area = bin_area_total - bin_area_used
    wastage_percentage = (wastage_area / bin_area_total) * 100 if bin_area_total > 0 else 0
    
    return {
        'total_bins': total_bins,
        'total_area': bin_area_total,
        'used_area': bin_area_used,
        'wastage_area': wastage_area,
        'wastage_percentage': wastage_percentage,
        'bin_groups': bin_groups
    }


In [14]:
def print_bin_analysis(bins):
    """Print formatted analysis of bin usage"""
    analysis = analyze_bin_usage(bins)
    
    print("\n=== BIN PACKING ANALYSIS ===")
    print(f"\nTotal Bins Used: {analysis['total_bins']}")
    print(f"Total Area of Used Bins: {analysis['total_area']:.3f} sqm")
    print(f"Area Utilized by Pieces: {analysis['used_area']:.3f} sqm")
    print(f"Wasted Area: {analysis['wastage_area']:.3f} sqm")
    print(f"Wastage Percentage: {analysis['wastage_percentage']:.2f}%")
    
    print("\n=== BIN GROUPS ANALYSIS ===")
    for bin_size, data in analysis['bin_groups'].items():
        print(f"\nBin Size {bin_size}mm:")
        print(f"  Number of Bins: {data['count']}")
        print(f"  Total Area: {data['total_area']:.3f} sqm")
        print(f"  Used Area: {data['used_area']:.3f} sqm")
        print(f"  Wastage: {((data['total_area'] - data['used_area']) / data['total_area'] * 100):.2f}%")
        
        # Group pieces by size
        piece_groups = {}
        for piece in data['pieces']:
            piece_key = f"{piece[0]}x{piece[1]}"
            piece_groups[piece_key] = piece_groups.get(piece_key, 0) + 1
        
        print("  Pieces Distribution:")
        for piece_size, count in piece_groups.items():
            print(f"    {piece_size}mm: {count} pieces")


In [15]:
def visualize_bins_simple(bins):
    """Create a simple visualization of bin usage"""
    # Calculate how many bins to show per row
    bins_per_row = min(3, len(bins))
    rows = (len(bins) + bins_per_row - 1) // bins_per_row
    
    fig, axes = plt.subplots(rows, bins_per_row, figsize=(15, 5*rows))
    if rows == 1 and bins_per_row == 1:
        axes = [[axes]]
    elif rows == 1 or bins_per_row == 1:
        axes = [axes]
        
    for i, bin in enumerate(bins):
        row = i // bins_per_row
        col = i % bins_per_row
        ax = axes[row][col]
        
        # Draw bin outline
        bin_width, bin_height = bin['size']
        ax.add_patch(patches.Rectangle((0, 0), bin_width, bin_height, 
                                     fill=False, color='black'))
        
        # Draw pieces with different colors
        colors = plt.cm.rainbow(np.linspace(0, 1, len(bin['placements'])))
        for piece, color in zip(bin['placements'], colors):
            ax.add_patch(patches.Rectangle((0, 0), piece[0], piece[1],
                                         color=color, alpha=0.5))
        
        ax.set_title(f'Bin {i+1}: {bin_width}x{bin_height}mm')
        ax.set_xlim(-50, bin_width + 50)
        ax.set_ylim(-50, bin_height + 50)
        
    plt.tight_layout()
    plt.show()

# Add these lines after your bin packing algorithm
print_bin_analysis(bins)
visualize_bins_simple(bins)


=== BIN PACKING ANALYSIS ===

Total Bins Used: 260
Total Area of Used Bins: 1331.148 sqm
Area Utilized by Pieces: 451.142 sqm
Wasted Area: 880.006 sqm
Wastage Percentage: 66.11%

=== BIN GROUPS ANALYSIS ===

Bin Size 2438x2100mm:
  Number of Bins: 260
  Total Area: 1331.148 sqm
  Used Area: 451.142 sqm
  Wastage: 66.11%
  Pieces Distribution:
    976x2358mm: 3 pieces
    967x2328mm: 21 pieces
    954x2268mm: 7 pieces
    951x2263mm: 7 pieces
    951x2218mm: 7 pieces
    954x2218mm: 7 pieces
    964x1902mm: 14 pieces
    973x1883mm: 6 pieces
    952x1738mm: 45 pieces
    973x1608mm: 12 pieces
    964x1603mm: 56 pieces
    1010x1594mm: 75 pieces
